# 🗄️ **Módulo 3: Vector Stores - A Memória Inteligente**

> *Onde guardar embeddings de forma que seja fácil encontrar depois*

---

## **Aula 3.1: Vector Stores Básicos - Onde Guardar as Informações**

---

### **Tá, mas o que são Vector Stores?**

Vector Stores são como **bibliotecas super inteligentes** que organizam informações não por ordem alfabética, mas por **similaridade de significado**. É tipo ter uma biblioteca onde livros sobre "gatos" ficam próximos de livros sobre "cachorros".

**🖼️ Sugestão de imagem**: Biblioteca organizada por temas, não por ordem alfabética

**Por que Vector Stores são importantes?**

Imagine que você tem 1 milhão de documentos e quer encontrar os 5 mais relevantes para uma pergunta:
- �� **Sem Vector Store**: Procurar em todos os 1 milhão (impossível!)
- 🗄️ **Com Vector Store**: Encontrar em milissegundos!

### **Analogia do Dia a Dia**

Vector Stores são como um **shopping center inteligente**:
- 🛍️ **Loja de roupas**: Todas as roupas ficam juntas
- �� **Praça de alimentação**: Todos os restaurantes ficam juntos
- �� **Área de lazer**: Cinemas e jogos ficam juntos

**Cada "área" tem itens similares, facilitando a busca!**

---

### **Setup Inicial - Preparando o Terreno**

**⚠️ IMPORTANTE**: Se você não executou o notebook `00_setup_colab.ipynb` primeiro, execute a célula abaixo para instalar as dependências.

In [ ]:
# 🚀 SETUP GRATUITO PARA COLAB
# Execute esta célula primeiro para configurar o ambiente!

# Instalando dependências (execute apenas se necessário)
!pip install langchain>=0.1.0
!pip install langchain-community>=0.0.10
!pip install langchain-core>=0.1.0
!pip install python-dotenv>=1.0.0
!pip install huggingface_hub>=0.19.0
!pip install sentence-transformers>=2.2.0
!pip install chromadb>=0.4.0
!pip install faiss-cpu>=1.7.0
!pip install numpy>=1.24.0
!pip install pandas>=2.0.0
!pip install matplotlib>=3.5.0
!pip install scikit-learn>=1.3.0

print("✅ Dependências instaladas com sucesso!")
print("🚀 Ambiente configurado para Vector Stores!")

In [ ]:
# 🗄️ IMPORTAÇÕES PARA VECTOR STORES
import numpy as np
import pandas as pd
import time
from datetime import datetime

# LangChain Vector Stores
from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores import Pinecone

# Embeddings
from langchain_community.embeddings import HuggingFaceEmbeddings

# Documentos
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Utilitários
import os
from dotenv import load_dotenv

print("✅ Bibliotecas importadas com sucesso!")
print("�� Pronto para explorar o mundo dos Vector Stores!")

### **Exemplo Prático - Vector Store Simples**

Vamos começar com um exemplo simples para entender o conceito:

In [ ]:
# �� EXEMPLO PRÁTICO: VECTOR STORE SIMPLES

# Criando documentos de exemplo
documentos_exemplo = [
    "O gato está dormindo no sofá da sala",
    "O cachorro está brincando no jardim da casa",
    "O peixe está nadando no aquário da sala",
    "O carro está estacionado na garagem",
    "A moto está na rua em frente à casa",
    "A pessoa está feliz com a notícia recebida",
    "A pessoa está triste com a perda do emprego",
    "O computador está ligado na mesa do escritório",
    "O celular está carregando na tomada",
    "O livro está aberto na página 50"
]

print("📚 DOCUMENTOS DE EXEMPLO:")
print("=" * 50)

for i, doc in enumerate(documentos_exemplo, 1):
    print(f"{i:2d}. {doc}")

print(f"\n📊 Total de documentos: {len(documentos_exemplo)}")

### **Configurando Embeddings**

Primeiro, vamos configurar os embeddings que vamos usar:

In [ ]:
# �� CONFIGURANDO EMBEDDINGS

def configurar_embeddings():
    """Configura modelo de embeddings"""
    
    try:
        print("🚀 Carregando Sentence Transformers...")
        
        embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2",
            model_kwargs={'device': 'cpu'}
        )
        
        print("✅ Embeddings configurados com sucesso!")
        return embeddings
        
    except Exception as e:
        print(f"⚠️ Erro ao carregar embeddings: {e}")
        print("💡 Vamos usar embeddings simulados para continuar")
        return None

# Configurando embeddings
embeddings = configurar_embeddings()

if embeddings:
    print(f"📊 Dimensão dos embeddings: {embeddings.client.get_sentence_embedding_dimension()}")
else:
    print("⚠️ Usando embeddings simulados para demonstração")

### **Criando Vector Store com Chroma**

Vamos criar nosso primeiro Vector Store usando Chroma (gratuito e local):

In [ ]:
# ��️ CRIANDO VECTOR STORE COM CHROMA

def criar_chroma_store(documentos, nome_colecao="minha_colecao"):
    """Cria um vector store usando Chroma"""
    
    print(f"🔧 Criando Vector Store Chroma: '{nome_colecao}'")
    
    if embeddings:
        # Usando embeddings reais
        vectorstore = Chroma.from_texts(
            texts=documentos,
            embedding=embeddings,
            collection_name=nome_colecao
        )
        print("✅ Vector Store criado com embeddings reais!")
    else:
        # Usando embeddings simulados
        vectorstore = Chroma.from_texts(
            texts=documentos,
            collection_name=nome_colecao
        )
        print("✅ Vector Store criado (modo simulado)!")
    
    return vectorstore

# Criando nosso primeiro vector store
print("�� CRIANDO PRIMEIRO VECTOR STORE:")
print("=" * 50)

chroma_store = criar_chroma_store(documentos_exemplo, "exemplo_colecao")

print(f"\n📊 Informações do Vector Store:")
print(f"   �� Documentos armazenados: {len(documentos_exemplo)}")
print(f"   🗄️ Tipo: Chroma (local e gratuito)")
print(f"   �� Persistente: Sim (salva no disco)")

### **Testando Busca no Vector Store**

Agora vamos testar como fazer buscas no nosso Vector Store:

In [ ]:
# 🔍 TESTANDO BUSCA NO VECTOR STORE

def testar_busca_vector_store(vectorstore, consultas_teste):
    """Testa diferentes tipos de busca no vector store"""
    
    print("�� TESTANDO BUSCAS NO VECTOR STORE:")
    print("=" * 50)
    
    for consulta in consultas_teste:
        print(f"\n❓ Consulta: '{consulta}'")
        
        # Medindo tempo de busca
        inicio = time.time()
        
        # Busca por similaridade
        resultados = vectorstore.similarity_search(consulta, k=3)
        
        tempo_busca = time.time() - inicio
        
        print(f"⏱️  Tempo de busca: {tempo_busca:.4f} segundos")
        print(f"📊 Resultados encontrados: {len(resultados)}")
        
        # Mostrando resultados
        for i, resultado in enumerate(resultados, 1):
            print(f"  {i}. {resultado.page_content}")
        
        print("-" * 40)

# Consultas para teste
consultas_teste = [
    "animal doméstico",
    "veículo motorizado",
    "dispositivo eletrônico",
    "emotion humana",
    "objeto na casa"
]

# Testando buscas
testar_busca_vector_store(chroma_store, consultas_teste)

### **Comparando Diferentes Vector Stores**

Vamos comparar Chroma com FAISS (outro vector store popular):

In [ ]:
# 📊 COMPARANDO DIFERENTES VECTOR STORES

def criar_faiss_store(documentos):
    """Cria um vector store usando FAISS"""
    
    print("🔧 Criando Vector Store FAISS...")
    
    if embeddings:
        # Usando embeddings reais
        vectorstore = FAISS.from_texts(
            texts=documentos,
            embedding=embeddings
        )
        print("✅ Vector Store FAISS criado com embeddings reais!")
    else:
        print("⚠️ FAISS requer embeddings reais")
        return None
    
    return vectorstore

def comparar_vector_stores(chroma_store, faiss_store, consultas_teste):
    """Compara performance entre diferentes vector stores"""
    
    print("📊 COMPARAÇÃO DE VECTOR STORES:")
    print("=" * 60)
    
    resultados_comparacao = {}
    
    # Testando Chroma
    print("\n�� Testando Chroma:")
    tempos_chroma = []
    
    for consulta in consultas_teste:
        inicio = time.time()
        resultados = chroma_store.similarity_search(consulta, k=3)
        tempo = time.time() - inicio
        tempos_chroma.append(tempo)
        
        print(f"  '{consulta}': {tempo:.4f}s")
    
    tempo_medio_chroma = np.mean(tempos_chroma)
    print(f"  ⏱️  Tempo médio Chroma: {tempo_medio_chroma:.4f}s")
    
    # Testando FAISS (se disponível)
    if faiss_store:
        print("\n🔍 Testando FAISS:")
        tempos_faiss = []
        
        for consulta in consultas_teste:
            inicio = time.time()
            resultados = faiss_store.similarity_search(consulta, k=3)
            tempo = time.time() - inicio
            tempos_faiss.append(tempo)
            
            print(f"  '{consulta}': {tempo:.4f}s")
        
        tempo_medio_faiss = np.mean(tempos_faiss)
        print(f"  ⏱️  Tempo médio FAISS: {tempo_medio_faiss:.4f}s")
        
        # Comparação
        print("\n�� COMPARAÇÃO FINAL:")
        print(f"  🗄️ Chroma: {tempo_medio_chroma:.4f}s")
        print(f"  🗄️ FAISS: {tempo_medio_faiss:.4f}s")
        
        if tempo_medio_faiss < tempo_medio_chroma:
            print(f"  🏆 FAISS é {tempo_medio_chroma/tempo_medio_faiss:.1f}x mais rápido!")
        else:
            print(f"  🏆 Chroma é {tempo_medio_faiss/tempo_medio_chroma:.1f}x mais rápido!")
    
    return resultados_comparacao

# Criando FAISS store
faiss_store = criar_faiss_store(documentos_exemplo)

# Comparando performance
if faiss_store:
    comparar_vector_stores(chroma_store, faiss_store, consultas_teste[:3])
else:
    print("\n�� Dica: FAISS é mais rápido para datasets grandes, mas requer embeddings reais")

## **Aula 3.2: Vector Stores Avançados - Otimizando Performance**

---

### **Tá, mas como otimizar Vector Stores para datasets grandes?**

Quando você tem milhões de documentos, precisa de estratégias inteligentes para manter a performance. É como organizar uma biblioteca gigante!

**🖼️ Sugestão de imagem**: Biblioteca gigante com sistema de organização inteligente

### **Setup Inicial - Preparando o Terreno**

Vamos criar um dataset maior para testar otimizações:

In [ ]:
# �� CRIANDO DATASET MAIOR PARA TESTES

def criar_dataset_grande(num_documentos=100):
    """Cria um dataset maior para testar performance"""
    
    print(f"📚 Criando dataset com {num_documentos} documentos...")
    
    # Templates para gerar documentos variados
    templates = [
        "O {animal} está {acao} no {local}",
        "A {pessoa} está {emocao} com {evento}",
        "O {objeto} está {estado} na {localizacao}",
        "A {empresa} está {situacao} no {mercado}",
        "O {produto} está {disponibilidade} na {loja}"
    ]
    
    # Vocabulário para preencher templates
    vocabulario = {
        "animal": ["gato", "cachorro", "peixe", "pássaro", "hamster", "coelho", "tartaruga"],
        "acao": ["dormindo", "brincando", "comendo", "correndo", "pulando", "nadando"],
        "local": ["sofá", "jardim", "quarto", "sala", "cozinha", "varanda", "escritório"],
        "pessoa": ["João", "Maria", "Pedro", "Ana", "Carlos", "Lucia", "Roberto"],
        "emocao": ["feliz", "triste", "animado", "preocupado", "surpreso", "calmo"],
        "evento": ["aniversário", "promoção", "viagem", "casamento", "formatura"],
        "objeto": ["computador", "celular", "livro", "carro", "moto", "bicicleta"],
        "estado": ["ligado", "desligado", "quebrado", "novo", "velho", "limpo"],
        "localizacao": ["mesa", "estante", "garagem", "escritório", "quarto"],
        "empresa": ["TechCorp", "InnoSoft", "DataFlow", "CloudTech", "SmartSys"],
        "situacao": ["crescendo", "expandindo", "inovando", "liderando", "investindo"],
        "mercado": ["tecnologia", "saúde", "educação", "finanças", "varejo"],
        "produto": ["smartphone", "laptop", "tablet", "smartwatch", "headphones"],
        "disponibilidade": ["disponível", "em estoque", "esgotado", "pré-venda"],
        "loja": ["shopping", "site online", "loja física", "marketplace"]
    }
    
    import random
    
    documentos_grandes = []
    
    for i in range(num_documentos):
        # Escolhendo template aleatório
        template = random.choice(templates)
        
        # Preenchendo template
        documento = template
        for placeholder, opcoes in vocabulario.items():
            if placeholder in documento:
                valor = random.choice(opcoes)
                documento = documento.replace(f"{{{placeholder}}}", valor)
        
        documentos_grandes.append(documento)
    
    print(f"✅ Dataset criado com {len(documentos_grandes)} documentos únicos!")
    
    # Mostrando alguns exemplos
    print("\n📝 Exemplos de documentos gerados:")
    for i in range(min(5, len(documentos_grandes))):
        print(f"  {i+1}. {documentos_grandes[i]}")
    
    return documentos_grandes

# Criando dataset maior
documentos_grandes = criar_dataset_grande(50)  # 50 documentos para teste

### **Otimizando Vector Store para Dataset Grande**

Vamos criar um Vector Store otimizado para o dataset maior:

In [ ]:
# 🚀 OTIMIZANDO VECTOR STORE PARA DATASET GRANDE

def criar_vector_store_otimizado(documentos, nome="otimizado"):
    """Cria um vector store otimizado para performance"""
    
    print(f"🚀 Criando Vector Store otimizado: '{nome}'")
    print(f"📊 Documentos: {len(documentos)}")
    
    inicio = time.time()
    
    if embeddings:
        # Usando FAISS para melhor performance em datasets grandes
        try:
            vectorstore = FAISS.from_texts(
                texts=documentos,
                embedding=embeddings
            )
            print("✅ Vector Store FAISS criado (otimizado para performance)!")
        except:
            # Fallback para Chroma
            vectorstore = Chroma.from_texts(
                texts=documentos,
                embedding=embeddings,
                collection_name=f"{nome}_colecao"
            )
            print("✅ Vector Store Chroma criado!")
    else:
        # Usando Chroma com embeddings simulados
        vectorstore = Chroma.from_texts(
            texts=documentos,
            collection_name=f"{nome}_colecao"
        )
        print("✅ Vector Store Chroma criado (modo simulado)!")
    
    tempo_criacao = time.time() - inicio
    print(f"⏱️  Tempo de criação: {tempo_criacao:.2f} segundos")
    
    return vectorstore

# Criando vector store otimizado
vector_store_grande = criar_vector_store_otimizado(documentos_grandes, "grande_dataset")

print(f"\n📊 Informações do Vector Store Otimizado:")
print(f"   📚 Documentos: {len(documentos_grandes)}")
print(f"   ��️ Tipo: {'FAISS' if 'FAISS' in str(type(vector_store_grande)) else 'Chroma'}")
print(f"   🚀 Otimizado para: Performance em datasets grandes")

### **Testando Performance com Dataset Grande**

Vamos testar a performance do Vector Store otimizado:

In [ ]:
# 📊 TESTANDO PERFORMANCE COM DATASET GRANDE

def testar_performance_vector_store(vectorstore, consultas_teste, num_resultados=5):
    """Testa performance do vector store com diferentes parâmetros"""
    
    print(f"📊 TESTE DE PERFORMANCE - {num_resultados} resultados por busca:")
    print("=" * 60)
    
    tempos_busca = []
    resultados_por_consulta = {}
    
    for consulta in consultas_teste:
        print(f"\n🔍 Consulta: '{consulta}'")
        
        # Medindo tempo
        inicio = time.time()
        resultados = vectorstore.similarity_search(consulta, k=num_resultados)
        tempo = time.time() - inicio
        
        tempos_busca.append(tempo)
        resultados_por_consulta[consulta] = resultados
        
        print(f"⏱️  Tempo: {tempo:.4f}s")
        print(f"📊 Resultados: {len(resultados)}")
        
        # Mostrando top 3 resultados
        for i, resultado in enumerate(resultados[:3], 1):
            print(f"  {i}. {resultado.page_content}")
    
    # Estatísticas
    tempo_medio = np.mean(tempos_busca)
    tempo_min = np.min(tempos_busca)
    tempo_max = np.max(tempos_busca)
    
    print(f"\n📈 ESTATÍSTICAS DE PERFORMANCE:")
    print(f"   ⏱️  Tempo médio: {tempo_medio:.4f}s")
    print(f"   ⏱️  Tempo mínimo: {tempo_min:.4f}s")
    print(f"   ⏱️  Tempo máximo: {tempo_max:.4f}s")
    print(f"   �� Total de buscas: {len(consultas_teste)}")
    
    return tempos_busca, resultados_por_consulta

# Consultas para teste de performance
consultas_performance = [
    "animal doméstico",
    "tecnologia empresa",
    "dispositivo eletrônico",
    "pessoa feliz",
    "produto disponível"
]

# Testando performance
tempos, resultados = testar_performance_vector_store(
    vector_store_grande, 
    consultas_performance, 
    num_resultados=5
)

### **Salvando e Carregando Vector Stores**

Vamos aprender como salvar e carregar Vector Stores para reutilização:

In [ ]:
# 💾 SALVANDO E CARREGANDO VECTOR STORES

def salvar_vector_store(vectorstore, nome_arquivo):
    """Salva um vector store para reutilização"""
    
    print(f"💾 Salvando Vector Store: '{nome_arquivo}'")
    
    try:
        if hasattr(vectorstore, 'save_local'):
            # FAISS
            vectorstore.save_local(nome_arquivo)
            print(f"✅ Vector Store salvo em: {nome_arquivo}")
        else:
            # Chroma salva automaticamente
            print(f"✅ Vector Store Chroma salvo automaticamente")
        
        return True
        
    except Exception as e:
        print(f"❌ Erro ao salvar: {e}")
        return False

def carregar_vector_store(nome_arquivo, embeddings_model=None):
    """Carrega um vector store salvo"""
    
    print(f"📂 Carregando Vector Store: '{nome_arquivo}'")
    
    try:
        if os.path.exists(nome_arquivo):
            # Tentando carregar FAISS
            vectorstore = FAISS.load_local(nome_arquivo, embeddings_model)
            print(f"✅ Vector Store FAISS carregado de: {nome_arquivo}")
        else:
            print(f"⚠️ Arquivo não encontrado: {nome_arquivo}")
            return None
        
        return vectorstore
        
    except Exception as e:
        print(f"❌ Erro ao carregar: {e}")
        return None

# Salvando nosso vector store
nome_arquivo = "meu_vector_store"
salvou = salvar_vector_store(vector_store_grande, nome_arquivo)

if salvou:
    # Carregando de volta
    vector_store_carregado = carregar_vector_store(nome_arquivo, embeddings)
    
    if vector_store_carregado:
        print("\n🧪 Testando vector store carregado:")
        resultado_teste = vector_store_carregado.similarity_search("animal", k=2)
        print(f"✅ Busca funcionando! Encontrados {len(resultado_teste)} resultados")
        for i, doc in enumerate(resultado_teste, 1):
            print(f"  {i}. {doc.page_content}")

print("\n💡 Dica: Vector Stores salvos podem ser reutilizados sem recriar embeddings!")

### **Teste Rápido**

Vamos fazer um teste rápido para verificar se tudo está funcionando:

In [ ]:
# 🧪 TESTE RÁPIDO: VECTOR STORES

def teste_rapido_vector_stores():
    """Teste rápido para verificar funcionamento dos vector stores"""
    
    print("🧪 TESTE RÁPIDO - VECTOR STORES:")
    print("=" * 50)
    
    # Teste 1: Chroma
    print("\n🔍 Teste 1: Chroma Store")
    try:
        resultado_chroma = chroma_store.similarity_search("gato", k=1)
        print(f"✅ Chroma funcionando: {resultado_chroma[0].page_content}")
    except Exception as e:
        print(f"❌ Erro no Chroma: {e}")
    
    # Teste 2: Vector Store Grande
    print("\n🔍 Teste 2: Vector Store Grande")
    try:
        resultado_grande = vector_store_grande.similarity_search("tecnologia", k=1)
        print(f"✅ Vector Store Grande funcionando: {resultado_grande[0].page_content}")
    except Exception as e:
        print(f"❌ Erro no Vector Store Grande: {e}")
    
    # Teste 3: Performance
    print("\n🔍 Teste 3: Performance")
    try:
        inicio = time.time()
        vector_store_grande.similarity_search("teste", k=5)
        tempo = time.time() - inicio
        print(f"✅ Performance OK: {tempo:.4f}s para busca com 5 resultados")
    except Exception as e:
        print(f"❌ Erro de performance: {e}")

# Executando teste rápido
teste_rapido_vector_stores()

### **Desafio do Módulo**

Agora é sua vez! Crie seu próprio Vector Store:

In [ ]:
# 🎯 DESAFIO DO MÓDULO

print("🎯 DESAFIO: Crie seu próprio Vector Store!")
print("=" * 40)
print("1. Adicione seus próprios documentos na lista abaixo")
print("2. Crie um vector store otimizado")
print("3. Teste buscas com suas consultas")
print("4. Salve o vector store para reutilização")
print("=" * 40)

# Seus documentos personalizados
meus_documentos = [
    # Adicione aqui documentos sobre um tema que você conhece
    # Exemplo: sua empresa, hobby, área de estudo, etc.
    "",
    "",
    "",
    "",
    ""
]

# Descomente e execute quando tiver seus documentos:
# if any(meus_documentos):
#     print("\n�� CRIANDO SEU VECTOR STORE:")
#     meu_vector_store = criar_vector_store_otimizado(meus_documentos, "meu_personalizado")
#     
#     print("\n🧪 TESTANDO SEU VECTOR STORE:")
#     minhas_consultas = ["sua consulta 1", "sua consulta 2"]
#     testar_performance_vector_store(meu_vector_store, minhas_consultas)
#     
#     print("\n💾 SALVANDO SEU VECTOR STORE:")
#     salvar_vector_store(meu_vector_store, "meu_vector_store_personalizado")

## **�� Módulo 3 Concluído!**

### **✅ O que aprendemos:**

1. **🗄️ O que são Vector Stores**: Bibliotecas inteligentes para embeddings
2. **🔍 Busca por similaridade**: Como encontrar documentos relacionados
3. **📊 Comparação de performance**: Chroma vs FAISS
4. **�� Otimização**: Como lidar com datasets grandes
5. **💾 Persistência**: Como salvar e carregar vector stores
6. **�� Performance**: Medindo e otimizando velocidade de busca

### **�� Próximos Passos:**

No próximo módulo vamos aprender sobre **Document Loaders** - como carregar documentos de diferentes fontes!

### **💡 Dicas Importantes:**

- **Chroma** é ótimo para desenvolvimento e datasets pequenos
- **FAISS** é melhor para datasets grandes e produção
- **Pinecone** é ideal para aplicações em nuvem
- **Sempre salve** vector stores para reutilização

---

**🎯 Desafio do Módulo**: Crie um Vector Store com seus próprios documentos!

**💡 Dica do Pedro**: Vector Stores são como GPS para documentos - sempre encontram o caminho mais rápido!

**🚀 Próximo módulo**: Document Loaders - Carregando tudo que existe!